# 1 Procesamiento Semántico de Palabras

## 1.1 Propósito

Se procesan una serie de textos y se obtiene el top k palabras más importantes y se exporta una lista para construir una nube de palabras a través de un programa externo.

## 1.2 Metodología

#### 1.2.1 Limpieza:
- Convierte caracteres de utf8 a ascii y elimina errores.
- Convierte mayúsculas en miinúsculas y quita acentos.
- Elimina filas con entradas vacías.
- Se deshace de palabras con un número de carácteres menor a 3.
- Elimina stop [words](https://nlp.stanford.edu/IR-book/html/htmledition/dropping-common-terms-stop-words-1.html)

#### 1.2.2 Vectorización TF-IDF

- Genera un vectorizador de textos usando tf-idf.
- Obtiene los vectores de importancia de palabra cada texto.


Tf-idf significa *Term Frequency - Inverse Term Frequency*, tf-idf es utilizado en la recuperación de información y minería de texto. TF-idf es una medida estadística utilizada para evaluar la importancia de una palabra para un documento en una colección
de textos o corpus.[2]

En un corpus de texto grande, algunas palabras están muy presentes
(por ejemplo, "el", "a", "y"), por lo tanto, llevarán muy poca información sobre el contenido real del
documento. Si tuviéramos que alimentar los datos de conteo directamente a un
clasificador, esos términos muy frecuentes tendrían mucho peso comparado
con el peso de términos más raros pero más interesantes.[3]

Usando la configuración predeterminada de [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html?highlight=tf%20idf#sklearn.feature_extraction.text.TfidfTransformer) de la libreriía Sci-Kit:

    TfidfTransformer(norm = 'l2', use_idf = True,
                     smooth_idf = True, sublinear_tf = False)

La frecuencia del término es decir, el número de veces que se produce un
término en un documento determinado, se multiplica con el componente idf, que
es calculado como:


>![Pipeline_inicial_sin_detalle](./assets/eq.png)

#### Ejemplo

    Considere un documento que contiene 100 palabras en el que la palabra gato
    aparece 3 veces. El término frecuencia (es decir, tf) para gato es
    (3/100) = 0.03. Ahora, supongamos que tenemos 10 millones de documentos y
    la palabra gato aparece en mil de estos. La frecuencia inversa del
    documento (es decir, idf) se calcula como log(10,000,001 / 1,000+1) = 4.
    Por lo tanto, el peso Tf-idf es el producto de estas cantidades:
    0.03 * 4 = 0.12.


A continuación se muestra la matríz TF-IDF de un conjunto de documentos de
diferentes temas tales como "air", "pollution", etc. con un conjunto de palabras
etiquetadas como d0, d1, dn:

>![Pipeline_inicial_sin_detalle](./assets/word_document.png)
   Matríz de words-documents, cada columna corresponde a un documento, cada fila a una palabra. Una celda almacena la frecuencia de una palabra en un documento, las celdas oscuras indican altas frecuencias de palabras. La clusterización por topic Modelling agrupan los
   documentos, que usan palabras similares, así como las palabras que aparecen
   en un conjunto similar de documentos. Los patrones resultantes se
   denominan "temas" o "tópicos".

  Obtenido de: [http://www.issac-conference.org/2015/Slides/Moitra.pdf](http://www.issac-conference.org/2015/Slides/Moitra.pdf)
  
Finalmente se  extraen las  top-25 palabras para generar la   nubede palabras y las top-20 para un  análisis externo.
  #####  Dependencias

Los siguientes módulos son necesarios para ejecutar el código:

- joblib==0.14.0
- nltk==3.4.5
- numpy==1.17.4
- pandas==0.25.3
- python-dateutil==2.8.1
- pytz==2019.3
- scikit-learn==0.21.3
- scipy==1.3.3
- six==1.13.0
- sklearn==0.0

## 1.4 Resultados

Se extrajo el contenido semántico de los conjuntos de palabras y se realizó la nube de palabras en https://www.wordclouds.com.

Responsable: Daniel Bustillos (juan.bustillos@itdp.org)


# 2 Configuraciones

## 2.1 Imports

In [1]:
#general

import pandas as pd
import sys
from joblib import dump, load
sys.path.append("./scripts") 

# limpieza y Steamming
from cleaning_steamming import CleanTools
import nltk
nltk.download('punkt')


# Análisis de importancia por TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

[nltk_data] Downloading package punkt to /home/jupyter-daniel-
[nltk_data]     bc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 2.2 Configuraciones

Ninguna

# 3 Definición de parámetros
definimos la columna a procesar

In [2]:
columna = "pregunta 3"
columns_not_duplicate = ["id"]
columns_to_clean = columna

# 4 Importación de datos

En esta sección cargue todos los datos que serán empleados en los análisis subsecuentes

In [3]:
df_encuesta = pd.read_csv("./data/respuestas-softlaubnch.csv")
df_encuesta["id"] = df_encuesta.index.tolist()
df_encuesta.head(3)

,pregunta 1,pregunta 2,pregunta 3,id
0,Con niveles de servicio por trayecto,Agregar datos de flujo de movilidad de celular...,Dividir empleos -población económicamente acti...,0
1,"Sí, estrategias de marketing para nuevas perso...",zonas rojas,rutas de transporte con accesibilidad universal,1
2,"Sí, para el análisis de tiempos de traslados y...",Incorporar líneas de transporte,Presencia de luminarias y de frentes activos,2


# 5 Procesamiento/Análisis/Modelos

### 5.1 Limpieza:


Separamos palabras en "/", util en campos normalizados "Porqué_norm" y "Limitación_norm"

In [4]:
df_encuesta[columna] = df_encuesta[columna].str.replace('/',' ')

Aplicamos el archivo de limipieza y steamming

In [5]:
ct = CleanTools()
df_proc = ct.text_cleaner(df_encuesta, columns_to_clean=columns_to_clean, columns_not_na="id")
field_clean = ct.stem_sentence_apply(limpiar=True,
                       columns_not_duplicate=columns_not_duplicate,
                       columns_to_clean=columns_to_clean)

...text_cleaner
...stop words


/home/jupyter-daniel-bc/nlp-toolkit-espanol/./scripts/cleaning_steamming.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  s = s.str.normalize('NFKD').str.encode(


Filtramos respuestas vacías

In [6]:
field_clean = [text for text in field_clean if text != ""]

#### Exportamos para nube de palabras

In [7]:
" ".join(field_clean)

'dividir empleos poblacion economicamente activa genero rutas transporte accesibilidad universal presencia luminarias frentes activos pueda visualizar niveles pobreza incidencia delictiva raiz poblacion cantidades establecer equipamiento hace falta diversas necesidades zonas atracciones poner capas guarderias supermercados lugares trabajo remunerado informacion infraestructura urbana informacion inv inegi indices inseguridad informacion detallada transporte datos accesibilidad personas movilidad reducida informacion espacios publicos inseguros manzana horario viajes informacion poblacional sector indicar rampas pavimento tactil facilidades accesibilidad universal botones panico auxiliar mujeres victimas via publica camaras seguridad rampas identificar utas transporte 100 accesibilidad mapear zona mujeres cargo cuidado poderlas incluir capa cercania botones panico emergencia tomar cuenta dimensiones enhorno publico acuerdo movilidad nersonas discapacidad analisis ingresos poblacion trab

In [8]:
texto_nube = " ".join(field_clean)
file2write=open("./data/procesado/palabras" + columna + ".txt",'w')
file2write.write(texto_nube)
file2write.close()

### 5.2 Vectorización TF-IDF

Aplicamos el vectorizador para conocer las top k palabras más importantes

In [9]:
vectorizer = TfidfVectorizer()
corpus_vector = vectorizer.fit_transform(field_clean)
feature_array = np.array(vectorizer.get_feature_names())
mean_words = corpus_vector.toarray().mean(axis=0)

Hacemos un nuevo dataframe y con la información de las palabras y su peso

In [10]:
df_words_weighted = pd.DataFrame(columns=["word", "weight_Avg"], index=[i for i in range(len(mean_words))])
df_words_weighted["word"] = feature_array
df_words_weighted["weight_Avg"] = mean_words
df_words_weighted = df_words_weighted.sort_values("weight_Avg", ascending=False).reset_index(drop=True)

In [11]:
df_words_weighted.head(25)

,word,weight_Avg
0,mujeres,0.063662
1,informacion,0.060717
2,incidencia,0.052378
3,delictiva,0.052378
4,accesibilidad,0.047948
5,rampas,0.045769
6,movilidad,0.045643
7,transporte,0.043922
8,viajes,0.039536
9,poblacion,0.035407


Guardamos

In [12]:
df_words_weighted = df_words_weighted.head(25)
df_words_weighted.to_csv("./data/procesado/topk_palabras_" + columna + ".csv", index=False)

### 5.3 Nube de palabras
La nube de palabras se realizó en

# 6 Referencias

[1] [https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html), visitado el 12/12/2019.


[2] [http://www.tfidf.com](http://www.tfidf.com), visitado el 12/12/2019.

[3] [https://scikit-learn.org/stable/modules/feature_extraction.html?highlight=tfid](https://scikit-learn.org/stable/modules/feature_extraction.html?highlight=tfidf), visitado el 12/12/2019.

[4] [https://mlexplained.com/2017/12/28/a-practical-introduction-to-nmf-nonnegative-matrix-factorization/](https://mlexplained.com/2017/12/28/a-practical-introduction-to-nmf-nonnegative-matrix-factorization/) , visitado el 11/12/2019.

[5] Blair, S.J., Bi, Y. & Mulvenna, M.D. Appl Intell (2019). https://doi.org/10.1007/s10489-019-01438-z
